In [ ]:
# @title Установка библиотек
!pip install fastapi # Установим FastAPI
!pip install uvicorn # Установим ASGI-сервер
!pip install python-multipart # Необходимая зависимость для FastAPI (для работы с данными отправленных форм на сайте)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%%writefile main.py
from typing_extensions import AsyncIterator
from fastapi import FastAPI, File, UploadFile
from io import BytesIO
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from fastapi.responses import PlainTextResponse
import keras
from pickle import load
# import os
import shutil
import json


app = FastAPI()

# scaler = 0
# model = 0

# Получение датафрейма
def get_df(path_df):
  headers = ['Date', 'Open_price', 'Highest_price', 'Lower_price', 'Close_price', 'Volume']
  df = pd.read_csv(path_df, header=None, names=headers, index_col='Date', parse_dates=['Date'])
  df.drop(columns=['Volume'], inplace=True)
  return df

# Функция для получения нормализатора
def get_scaler(path):
  scaler = load(open(path, 'rb'))

  # Возвращаю нормализатор
  return scaler


# Разбиение на выборку
def get_x_test(df, scaler):
  test_data = df[:31]
  scaled_test_data = scaler.transform(test_data)
  n_input = 30
  tester = TimeseriesGenerator(scaled_test_data, scaled_test_data, length=n_input, batch_size=scaled_test_data.shape[0])
  x_test, y_test = tester[0]
  return x_test

# Функция расчета результата предсказания
def get_pred(model, # модель
             x_test,
            #  y_test, # тестовая выборка
             y_scaler): # масштабирующий объект для y

  # Вычисление и деномализация предсказания
  y_pred_unscaled = y_scaler.inverse_transform(model.predict(x_test, verbose=0))

  # Возврат результата предсказания
  return y_pred_unscaled

# Это было раньше для подготовки модели
# Пришлось все в метод post перенести
@app.get("/prepare_model")
def prepare_model():
  # Путь на сервере для загрузки модели


  try:
    # Загружаю модель


    return {"Получилось": "Модель подготовлена"}

  except Exception as e:
    return {"Неполучилось": e.args}




# @app.get("/train_scaler", response_class = PlainTextResponse)
# async def upload_train(file: UploadFile):
#   return {"filename": file.filename}
  # # Распаковка предобученной модели
  # !unzip '16_model.zip'

  # # Получение модели
  # model = keras.models.load_model('16_model')
  # scaler = get_scaler(file)
  # return 'Получилось'



@app.post("/upload")
def upload(file: UploadFile = File(...)):
  # Сохраняю файл
  try:
    os.mkdir("Client")
    file_path = f"./Client/{file.filename}"
    with open(file_path, "wb") as f:
      f.write(file.file.read())
      file.close()
      file.file.close()

    # Получаю датафрейм
    df = get_df(file_path)


    # Путь на сервере для загрузки scaler-а
    path_scaler = './Server/scaler.pkl'

    # Получаю нормализаор
    scaler = get_scaler(path_scaler)

    # Получаю выборку
    x_test = get_x_test(df, scaler)

    # Путь до предобученной модели
    path_model = './Server/load_model.keras'

    # Загружаю модель
    model = keras.models.load_model(path_model)

    # Предсказываю
    y_pred = get_pred(model, x_test, scaler)

    # Удаляю папку
    shutil.rmtree("./Client")

    return json.dumps(y_pred.tolist())

  except Exception as e:
    return {"message": e.args}
  # try:
  #   df = get_df(file)
  #   x_test = get_x_test(df)
  #   y_pred = get_pred(model, x_test, scaler)
  #   return 'yes'
  # except Exception as e:
  #   return {'Ошибка': e.args}
  # return {"message": f"y_pred = {y_pred}"}


Overwriting main.py


In [ ]:
!nohup uvicorn main:app --reload &

nohup: appending output to 'nohup.out'


In [ ]:
import requests
response = requests.get('http://127.0.0.1:8000/prepare_model')
print(response.status_code)
print(response.text)

200
{"Получилось":"Модель подготовлена"}


In [ ]:
import requests
myurl = 'http://127.0.0.1:8000/upload'
files = {'file': open('./1.csv', 'rb')}
getdata = requests.post(myurl, files=files)
print(f'Предсказание: {getdata.text}')

Предсказание: "[[41.76048278808594, 42.46329116821289, 41.11382293701172, 42.07912063598633]]"


In [ ]:
!ssh -o "StrictHostKeyChecking no" -R 80:localhost:8000 serveo.net
# !ssh -o "StrictHostKeyChecking no" -R 80:localhost:8000 nokey@localhost.run

Forwarding HTTP traffic from https://497d545897e7c1625a0a2ca9e40b1965.serveo.net
HTTP request from 217.22.172.178 to https://497d545897e7c1625a0a2ca9e40b1965.serveo.net/
HTTP request from 217.22.172.178 to https://497d545897e7c1625a0a2ca9e40b1965.serveo.net/favicon.ico
HTTP request from 217.22.172.178 to https://497d545897e7c1625a0a2ca9e40b1965.serveo.net/docs
HTTP request from 217.22.172.178 to https://497d545897e7c1625a0a2ca9e40b1965.serveo.net/docs
